In [1]:
import pandas as pd
import bs4 as bs
import re
import requests
from selenium import webdriver
from tqdm import tqdm_notebook as tqdm
from time import sleep
from os import listdir
from os.path import isfile, join, isdir
import random
import itertools
from pywebcopy import save_webpage

## Original Search Results

In [47]:
# import list of search strings
search_list = pd.read_csv('./sample_list.csv', skiprows=2)

In [93]:
# change spaces to %20 as prescribed in the new search url and store on list
query_list = search_list['Query'].str.replace(' ', '%20')
query_list = list(set(query_list))

In [94]:
# define query url
query_url = 'https://forum.cirruspilots.org/search?q='

In [2]:
# define site login function
def site_login():
    '''
    This is a helper function that logins to cirruspilots.org
    '''
    driver.get('https://passport.cirruspilots.org/')
    driver.find_element_by_id(
        'dnn_ctr_Login_Login_DNN_txtUsername').send_keys('jloretizo')
    driver.find_element_by_id(
        'dnn_ctr_Login_Login_DNN_txtPassword').send_keys('cth4h#@EoY*a')
    driver.find_element_by_id('login_btn').click()

In [18]:
# define the driver and execute login 
driver = webdriver.Safari()
site_login()

In [337]:
# get the results set
results_set = {}
for i in tqdm(query_list):
    driver.get(query_url + i, )
    sleep(2)
    soup = bs.BeautifulSoup(driver.page_source, 'html.parser')
    links_list = []
    for j in soup.find_all(class_='fps-result'):
        href = j.find(class_='search-link')['href']
        links_list.append(href)
    results_set[i] = links_list
    sleep(2)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


In [10]:
def get_post_ids(row):
    ref_nums = []
    for i in range(len(row)):
        try:
            num = re.findall(r'\/[0-9]+\/[0-9]|\/[\d]+$', str(row[i]))[0]
        except:
            num = None
        ref_nums.append(num)
    return ref_nums

In [370]:
results_df = pd.DataFrame.from_dict(results_set, orient='index')
# results_df = results_df.reset_index()
results_df['ref_nums'] = results_df.apply(
    lambda row: get_post_ids(row), axis=1)

In [412]:
full_links = results_df.drop('ref_nums', axis=1).stack().reset_index().rename(
    columns={'level_0': 'search string', 0: 'id'})

In [413]:
full_links['search string'] = full_links['search string'].apply(
    lambda x: x.replace('%20', ' '))
full_links.to_csv('./results_df.csv')

In [371]:
# converting links to dictionary
links_dict = {}
for i in results_df.index:
    links_dict[i] = results_df.loc[i]['ref_nums']

In [388]:
# stacking and resetting new dataframe
link_ids = pd.DataFrame.from_dict(
    links_dict, orient='index').stack().reset_index()

In [389]:
# column rename and search string renames
link_ids['level_0'] = link_ids['level_0'].apply(
    lambda x: x.replace('%20', ' '))
link_ids = link_ids.rename(columns={'level_0': 'search string', 0: 'id'})

In [393]:
# saving data to csv
link_ids[['search string', 'id']].to_csv('./search_results.csv', index=False)

## New Search Results

In [2]:
test_links = pd.read_csv('./test_links.csv')

In [3]:
# get all files from path
mypath = '/Users/jeloretizo/Documents/MSBA/Practicum Project/COPA-Usability-Testing'
files = [f for f in listdir(mypath) if isfile(join(mypath, f)) and ('ratio' in f)]

In [10]:
query_strings = ['brakes during taxi',
                 'capture glide slope above',
                 'stall under hood',
                 'find flight instructor',
                 'oxygen altitude',
                 'file nasa report',
                 'loose fairing noise',
                 'loose seat belt',
                 'power off landing',
                 'music volume low']

In [5]:
panel_labels = ['Panel1.html', 'Panel2.html', 'Panel3.html', 'Panel4.html']

In [31]:
def get_rerank_list(soup, rerank):
    ''' Helper function to rerank soup files'''
    all_items = soup.find_all(class_='fps-result')
    html_links = pd.DataFrame(all_items).reset_index()
    html_links = html_links.rename(columns={'index':'old_rank'})
    html_links['id'] = html_links[3].apply(lambda x: get_href(x))
    rerank = rerank.reset_index(drop=True).reset_index()
    rerank = rerank.rename(columns={'index': 'New Rank'})
    for i in html_links['id']:
        try: 
#             print(rerank.loc[temp_var['url-id'].str.contains(i)])
            new_rank = rerank.loc[rerank['url-id'].str.contains(i)].index[0]
        except:
            new_rank = None
        html_links.loc[html_links['id'] == i, 'New Rank'] = new_rank
#     print(html_links.head())
    new_order = list(html_links.sort_values('New Rank', ascending=True)['old_rank'])
#     print(html_links.head)
    final_order = [all_items[i] for i in new_order]
    return final_order

In [7]:
def get_id(item):
    ''' Helper function to get id from ratio combo file '''
    res_url = re.findall(r'\/[0-9]+\/[0-9]+|\/[\d]+$',item)[0]
    return res_url

In [8]:
def get_href(html):
    ''' Helper function to get id from the html files '''
    href = html.find(class_='search-link')['href']
    return get_id(href)

In [33]:
for i in tqdm(query_strings):
#     print(i)
    random_panels = random.sample(panel_labels, 4)
    folder_name = i.title().replace(' ', '')
    query_path = mypath + '/' + folder_name + '/'
    file = [f for f in listdir(query_path) if (
        i in f) and (isdir(join(query_path, f)) == False)]
    if len(file) == 1:
        with open('./{}/{}'.format(folder_name, file[0]), 'r') as open_file:
            soup = bs.BeautifulSoup(open_file, 'html.parser')
            saved_soup = soup
            print(i, ' -- Control Panel Printed in:', random_panels[3])
            with open('./{}/{}'.format(folder_name, random_panels[3]), 'w') as output_file:
                test_links.loc[(test_links['0'] == folder_name) & (test_links['1'] == random_panels[3]), 
                              'Panel Type'] = 'Control'
                output_file.write(str(saved_soup))
                output_file.close()
            model_number = 0
            for j in files:
                ratio = pd.read_csv('./' + j)
                ratio['url-id'] = ratio['url'].apply(lambda x: get_id(x))
                temp_var = ratio.loc[ratio['Query'] == i]
                final_list = get_rerank_list(soup, temp_var)
#                 print(final_list)
                with open('./{}/{}'.format(folder_name, file[0]), 'r') as open_file:
                    soup = bs.BeautifulSoup(open_file, 'html.parser')
                for k in range(len(final_list)):
                    soup.find_all(
                        class_='fps-result')[k].replace_with(final_list[k])
                print(i, ' -- Model Ratio: ', j, ' -- Printed in: ',
                      random_panels[model_number])
                test_links.loc[(test_links['0'] == folder_name) & (test_links['1'] == random_panels[model_number]), 
                              'Panel Type'] = j
                with open('./{}/{}'.format(folder_name, random_panels[model_number]), 'w') as output_file:
                    output_file.write(str(soup))
                    output_file.close()
                model_number += 1

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.


brakes during taxi  -- Control Panel Printed in: Panel1.html
brakes during taxi  -- Model Ratio:  ratio_comb3.csv  -- Printed in:  Panel2.html
brakes during taxi  -- Model Ratio:  ratio_comb2.csv  -- Printed in:  Panel3.html
brakes during taxi  -- Model Ratio:  ratio_comb1.csv  -- Printed in:  Panel4.html
capture glide slope above  -- Control Panel Printed in: Panel2.html
capture glide slope above  -- Model Ratio:  ratio_comb3.csv  -- Printed in:  Panel3.html
capture glide slope above  -- Model Ratio:  ratio_comb2.csv  -- Printed in:  Panel4.html
capture glide slope above  -- Model Ratio:  ratio_comb1.csv  -- Printed in:  Panel1.html
stall under hood  -- Control Panel Printed in: Panel2.html
stall under hood  -- Model Ratio:  ratio_comb3.csv  -- Printed in:  Panel1.html
stall under hood  -- Model Ratio:  ratio_comb2.csv  -- Printed in:  Panel3.html
stall under hood  -- Model Ratio:  ratio_comb1.csv  -- Printed in:  Panel4.html
find flight instructor  -- Control Panel Printed in: Panel3

In [34]:
test_links.to_csv('./test_links.csv', index=False)

In [15]:
query_strings[1]

'capture glide slope above'

In [137]:
test_links = pd.DataFrame(list(itertools.product(folder_names, panel_labels)))

In [138]:
test_links['github_link'] = 'https://johnelmer.github.io/COPA-Usability-Testing/' + test_links[0] + '/' + test_links[1]

In [147]:
def make_tiny(url): 
    request_url = ('http://tinyurl.com/api-create.php?' + urlencode({'url':url}))     
    with contextlib.closing(urlopen(request_url)) as response:                       
        return response.read().decode('utf-8 ')  

In [146]:
from __future__ import with_statement

import contextlib

try:
    from urllib.parse import urlencode

except ImportError:
    from urllib import urlencode

try:
    from urllib.request import urlopen

except ImportError:
    from urllib2 import urlopen

import sys

In [149]:
test_links['tiny_url'] = test_links['github_link'].apply(lambda x: make_tiny(x))

In [134]:
for i in itertools.product(folder_names, panel_labels):
    print(i)

('BrakesDuringTaxi', 'Panel1.html')
('BrakesDuringTaxi', 'Panel2.html')
('BrakesDuringTaxi', 'Panel3.html')
('BrakesDuringTaxi', 'Panel4.html')
('CaptureGlideSlopeAbove', 'Panel1.html')
('CaptureGlideSlopeAbove', 'Panel2.html')
('CaptureGlideSlopeAbove', 'Panel3.html')
('CaptureGlideSlopeAbove', 'Panel4.html')
('StallUnderHood', 'Panel1.html')
('StallUnderHood', 'Panel2.html')
('StallUnderHood', 'Panel3.html')
('StallUnderHood', 'Panel4.html')
('FindFlightInstructor', 'Panel1.html')
('FindFlightInstructor', 'Panel2.html')
('FindFlightInstructor', 'Panel3.html')
('FindFlightInstructor', 'Panel4.html')
('OxygenAltitude', 'Panel1.html')
('OxygenAltitude', 'Panel2.html')
('OxygenAltitude', 'Panel3.html')
('OxygenAltitude', 'Panel4.html')
('FileNasaReport', 'Panel1.html')
('FileNasaReport', 'Panel2.html')
('FileNasaReport', 'Panel3.html')
('FileNasaReport', 'Panel4.html')
('LooseFairingNoise', 'Panel1.html')
('LooseFairingNoise', 'Panel2.html')
('LooseFairingNoise', 'Panel3.html')
('LooseFa

In [37]:
# find all content item classes
for i in soup.find_all(class_='content-item'):
    # change the value of title inside the a tag
    i.find(class_='abbreviated-post').find('h4').find('a')['title'] = 'New Title'
    # change the href for link
    i.find(class_='abbreviated-post').find('h4').find('a')['href'] = 'New Title'
    # change text value
    i.find(class_='abbreviated-post').find('h4').find('a').string.replace_with("No longer bold")
    

In [38]:
# save this new html as test panel
with open("output1.html", "w") as file:
    file.write(str(soup))

In [41]:
soup.find_all(class_='content-item')[0].find('h4')

<h4 class="post-name"><a class="internal-link view-post" href="https://www.cirruspilots.org/copa/safety_programs/f/349/t/159914.aspx" title="New Title">No longer bold</a></h4>